In [1]:
from skimage.feature import local_binary_pattern
import cv2
import numpy as np
import pandas as pd

In [2]:
import traceback

try:
    def calculate_uniformity(lbp_image):
        # Ubah tipe data citra LBP menjadi integer dengan menghitung nilai integer terdekat
        # lbp_image = np.rint(lbp_image)
        lbp_image = np.rint(lbp_image).astype(int)


        # Hitung jumlah kemunculan setiap nilai LBP
        histogram = np.bincount(lbp_image.flatten())

        # Hitung uniformity dengan menjumlahkan setiap elemen pada histogram yang bernilai 1
        uniformity = sum([histogram[i] for i in range(len(histogram)) if histogram[i] == 1])

        return uniformity

    def calculate_energy(lbp_image):

        lbp_image = np.rint(lbp_image).astype(int)

        # Hitung histogram LBP
        histogram = np.bincount(lbp_image.flatten())

        # Hitung energy dengan menghitung sum-of-squares dari histogram
        energy = np.sum(histogram ** 2)

        return energy

    def calculate_entropy(lbp_image):

        lbp_image = np.rint(lbp_image).astype(int)

        # Hitung histogram LBP
        histogram = np.bincount(lbp_image.flatten())

        # Hitung probabilitas setiap nilai LBP
        prob = histogram / np.sum(histogram)

        # Hitung entropy dengan menghitung -sum(p * log(p)) untuk setiap nilai LBP
        entropy = -np.sum(prob * np.log2(prob))

        return entropy
except Exception as e:
    print(traceback.format_exc())

In [4]:
import os
import traceback

# Dictionary yang menyimpan label untuk setiap nama folder gambar
labels = {'SAPI': 'SAPI', 'BABI': 'BABI'}

# Direktori yang berisi sekumpulan folder gambar
image_dir = '/content/Dataset2'

# Ambil nama-nama folder di dalam direktori
folder_names = os.listdir(image_dir)

# Buat list kosong untuk menyimpan atribut LBP
uniformities = []
energies = []
entropies = []

# Buat list kosong untuk menyimpan label
image_labels = []

for folder_name in folder_names:
  # Ambil nama-nama file gambar di dalam folder
  image_filenames = os.listdir(os.path.join(image_dir, folder_name))

  for filename in image_filenames:
    # Baca gambar
    image = cv2.imread(os.path.join(image_dir, folder_name, filename))

    # Ubah citra ke skala abu-abu
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Buat objek LBP
    lbp_image = local_binary_pattern(gray_image, 8, 1, method='uniform')

    # Hitung atribut LBP
    uniformity = calculate_uniformity(lbp_image)
    energy = calculate_energy(lbp_image)
    entropy = calculate_entropy(lbp_image)

    # Tambahkan atribut LBP ke dalam list
    uniformities.append(uniformity)
    energies.append(energy)
    entropies.append(entropy)
    

    # Tambahkan label ke dalam list
    # image_labels.append(labels[folder_name])
    # Tambahkan label ke dalam list
    if folder_name in labels:
      image_labels.append(labels[folder_name])
    else:
      image_labels.append('unknown')

from sklearn import preprocessing

# Buat objek scaler
scaler = preprocessing.MinMaxScaler()

# Transformasikan nilai atribut menggunakan scaler
uniformities_scaled = scaler.fit_transform(np.array(uniformities).reshape(-1, 1))
energies_scaled = scaler.fit_transform(np.array(energies).reshape(-1, 1))
entropies_scaled = scaler.fit_transform(np.array(entropies).reshape(-1, 1))


# Buat data frame dengan atribut LBP dan label
# df = pd.DataFrame({'Uniformity': uniformities_scaled, 'Energy': energies_scaled, 'Entropy': entropies_scaled, 'Label': image_labels})
df = pd.DataFrame({'Uniformity': uniformities_scaled.flatten(), 'Energy': energies_scaled.flatten(), 'Entropy': entropies_scaled.flatten(), 'Label': image_labels})

# Simpan file CSV
df.to_csv('Dataset_LBP.csv', index=False)